In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
def train_test_split(df, test_size=0.2, random_state=42, split_by_k = True):
        df['date'] = df.index.date
        INDEX = df[['site_name','k']].copy()
        INDEX['date']= INDEX.index.date
        INDEX = INDEX.groupby(by=[df.index.date,'site_name']).mean()
        INDEX.reset_index(level=1, inplace=True)
        
        if split_by_k :
            klow = INDEX[INDEX.k<=0.3]
            kmed = INDEX[(INDEX.k>0.3) & (INDEX.k<=0.6)]
            khigh =  INDEX[(INDEX.k>0.6)]
            klow_test = klow.sample(frac=test_size,random_state=random_state)
            kmed_test = kmed.sample(frac=test_size,random_state=random_state)
            khigh_test = khigh.sample(frac=test_size,random_state=random_state)
            X_test = pd.concat([klow_test, kmed_test, khigh_test])

        else:
            X_test = INDEX.sample(frac=test_size, random_state = random_state)
            
        X_test.reset_index(inplace=True)
        X_test = X_test.rename(columns={'index':'date'})
        X_test = X_test.drop(columns = ['k'])
        X_test = df.reset_index().merge(X_test,on=['date','site_name']).set_index('Datetime')
        X_train = pd.concat([df,X_test]).drop_duplicates(keep=False)
        X_train.drop(columns=['date']); X_test.drop(columns=['date'])
        
        return X_train, X_test

In [7]:
TRIAN_FOLDER = 'C:\\Users\\Tee\\senior_project\\src\\code\\data\\training_data'
CM_DATA = 'DATASET_cloudmask.csv'

In [8]:
df = pd.read_csv(os.path.join(TRIAN_FOLDER,CM_DATA),parse_dates=['Datetime'],index_col='Datetime')

In [9]:
X_train, X_test = train_test_split(df, test_size=0.2, random_state=42, split_by_k = False)

In [10]:
proc = pd.concat([X_train, X_test])

In [11]:
#proc.to_csv('separated_DATASET_cloudmask.csv') # export data